# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

## Задачи для совместного разбора

1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.

In [ ]:
import re
obj = {
    "home_page": "https://github.com/pypa/sampleproject",
    "keywords": "sample setuptools development",
    "license": "MIT",
}

ind = len(max(obj.keys(), key=len))
for k, v in obj.items():
  print(k + f'{{:>{ind - len(k) + 3}}} '.format('=') + f'"{v}"')

home_page  = "https://github.com/pypa/sampleproject"
keywords   = "sample setuptools development"
license    = "MIT"


2. Написать регулярное выражение,которое позволит найти номера групп студентов.

In [ ]:
import re
import pandas as pd

obj = pd.Series(["Евгения гр.ПМ19-1", "Илья пм 20-4", "Анна 20-3"])
tab = pd.DataFrame(obj, columns = ['Name'])
tab['Number'] = tab.Name.apply(lambda x: re.search("\d{2}-\d", x).group())
tab

,Name,Number
0,Евгения гр.ПМ19-1,19-1
1,Илья пм 20-4,20-4
2,Анна 20-3,20-3


3. Разбейте текст формулировки задачи 2 на слова.

In [ ]:
import re
text = 'Написать регулярное выражение,которое позволит найти номера групп студентов'
re.findall('\w+', text)

['Написать',
 'регулярное',
 'выражение',
 'которое',
 'позволит',
 'найти',
 'номера',
 'групп',
 'студентов']

## Лабораторная работа 6

### Форматирование строк

1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны. 

In [ ]:
recipes = pd.read_csv('recipes_sample.csv')
recipes_random = recipes.sample(n=5)[['id', 'minutes']]
len_id = len(str(max(recipes_random.id))) + 5
len_min = len(str(max(recipes_random.minutes))) + 5
print(f"|{'id':^{len_id}}" + f"|{'minutes':^{len_min}}|")
print("|" + "-"*(len_id + len_min+1) + "|")
for i in recipes_random.index:
  print(f"|{recipes_random.loc[i, 'id']:^{len_id}}|" + f"{recipes_random.loc[i, 'minutes']:^{len_min}}|")

|    id     |minutes|
|-------------------|
|   82065   |  26   |
|  125967   |   6   |
|  482038   |  60   |
|   53203   |  40   |
|   32059   |  35   |


2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [ ]:
from bs4 import BeautifulSoup
with open('/content/steps_sample.xml') as f:
  fail = f.read()
sts = BeautifulSoup(fail, 'xml')
recipes.loc[recipes.id==170895]

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
15754,leeks and parsnips sauteed or creamed,170895,27,8377,2006-05-31,21.0,this is good sauteed only or creamed. very eas...,9.0


In [ ]:
r = sts.find_all('recipe')
def show_info(id):
  cur = recipes.loc[recipes.id==170895]
  for recipe in r:
    if recipe.find('id').text == str(id):
      steps = recipe.find_all('step')
      n = 1
      l_s = 0
      print(f"'{cur.name.values[0]}'")
      for step in steps:
        print(f"{n}. " + step.text)
        n += 1
        l_s = max(l_s, len(step.text))
  print('-' * (l_s+3))
  print(f"Автор: {cur.contributor_id.values[0]}")
  print(f"Среднее время приготовления: {cur.minutes.values[0]} минут")

show_info(170895)

'leeks and parsnips  sauteed or creamed'
1. clean the leeks and discard the dark green portions
2. cut the leeks lengthwise then into one-inch pieces
3. melt the butter in a medium skillet , med
4. heat
5. add the garlic and fry 'til fragrant
6. add leeks and fry until the leeks are tender , about 6-minutes
7. meanwhile , peel and chunk the parsnips into one-inch pieces
8. place in a steaming basket and steam 'til they are as tender as you prefer
9. i like them fork-tender
10. drain parsnips and add to the skillet with the leeks
11. add salt and pepper
12. gently sautee together for 5-minutes
13. at this point you can serve it , or continue on and cream it:
14. in a jar with a screw top , add the half-n-half and arrowroot
15. shake 'til blended
16. turn heat to low under the leeks and parsnips
17. pour in the arrowroot mixture , stirring gently as you pour
18. if too thick , gradually add the water
19. let simmer for a couple of minutes
20. taste to adjust seasoning , probably an addit

## Работа с регулярными выражениями

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [ ]:
recipes1 = sts.find_all('recipe')
def pattern(id):
  pattern = '\d+\s(?:hour|hours|minute|minutes)'
  for recipe in recipes1:
    if int(recipe.find('id').text) == id:
      steps = recipe.find_all('step')
      for step in steps:
        t = re.findall(pattern, step.text)
        if t:
          for i in t:
            print(i)
pattern(25082)

20 minute
10 minute
2 hour
10 minute
20 minute
30 minute


4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [ ]:
recipes = pd.read_csv('/content/recipes_sample.csv')

In [ ]:
patt = 'this[\w\s]*,(?:but| but)'
r = recipes.loc[recipes.description.str.match(patt)==True]
print(f"Количество рецептов: {r.shape[0]}\n")
for rec in range(1,4):
  print(f"Рецепт №{rec}: {r.description.values[rec]}")

Количество рецептов: 134

Рецепт №1: this was adapted from a recipe i found on the net, but i added julienne onion to the peppers.  this is a meal in itself, or you could have a small slice with a meat dish.  for those that like to have brunch, it's a bit different to your traditional quiche recipes.  if you love cheese, you could add 1/2 cup of your favorite to the egg mixture, then pour over peppers.
Рецепт №2: this is kind of similar to some of the other versions out there, but it is the best and easiest i have found
Рецепт №3: this is a moist, buttery apple cake.  the dough is best made the night before, put in the fridge and then filled and baked the next morning.


5\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [ ]:
r = sts.find_all('recipe')
def pattern2(id):
  pattern = '\d+ \/ \d+'
  for recipe in r:
    if int(recipe.find('id').text) == id:
      steps = recipe.find_all('step')
      for step in steps:
        step = step.text
        rez = re.search(pattern, step)
        if rez:
          rez2 = rez.group().split(' / ')
          step = re.sub(pattern, f"{rez2[0]}/{rez2[1]}", step)
        print(step)

pattern2(72367)

mix butter , flour , 1/3 c
sugar and 1-1/4 t
vanilla
press into greased 9" springform pan
mix cream cheese , 1/4 c
sugar , eggs and 1/2 t
vanilla beating until fluffy
pour over dough
combine apples , 1/3 c
sugar and cinnamon
arrange on top of cream cheese mixture and sprinkle with almonds
bake at 350 for 45-55 minutes , or until tester comes out clean


### Сегментация текста

6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [ ]:
pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
r = sts.find_all('recipe')
def wordF():
  words = set()
  for recipe in r:
    steps = recipe.find_all('step')
    for step in steps:
      step = step.text
      words.update(nltk.word_tokenize(step))
  words2 = set()
  for word in words:
    if word.isalpha():
      words2.add(word)
  return words2
  
print(len(wordF()))

14926


7\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [ ]:
def sentence():
  ids = [i for i in recipes.id.values[:5]]
  l_sent = [0 for i in range(5)]
  for recipe in r:
    id = recipe.find('id').text
    steps = recipe.find_all('step')
    l_s = 0
    for step in steps:
      step = step.text
      l_s = len(nltk.sent_tokenize(step))
    for i in range(len(l_sent)):
      if l_s > l_sent[i]:
        ids[i] = int(id)
        l_sent[i] = l_s
        break
  return ids, l_sent

ids, l_sent = sentence()
recipes.loc[recipes.id.isin(ids)]

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
10677,fabulous honey mustard chicken,115105,50,205967,2005-04-02,9.0,beautiful on a warm summers day served with a ...,NaN
13440,healthy breakfast bars my way,385874,90,381180,2009-08-17,10.0,a moist (adult) breakfast bar chocked full of ...,13.0
13822,home style beef n noodles w mushrooms onions,138239,135,113941,2005-09-21,NaN,"wish you could smell my kitchen right now, i'm...",14.0
14329,hungarian style cabbage rolls soooo good,151074,380,76503,2006-01-09,12.0,these are some of the best cabbage rolls i hav...,6.0
21718,pumpkin nuggets with vanilla sauce,391047,30,1375473,2009-09-21,NaN,i think few adore sugar and pumpkin as much as...,13.0
